# OUT OF CORE
### Crecacion de models con conjuntos de datos muy grandes

In [81]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ingva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [82]:
def tokenizer(text):
    """
    Se crea la funcion que tokenizara por medio de la funcion split
    """
    text = re.sub('<[^>]*>','',text)
    emo = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = (re.sub('[\W]+','',text.lower())+' '.join(emo).replace('-',''))
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [83]:
def stream_docs(path):
    """
    Permite extraer linea a linea los docs del file
    """
    with open(path,'r',encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [84]:
#Prueba de stream_docs
next(stream_docs('movie_data.csv'))

('"The infamous Ed Wood ""classic"" Plan 9 From Outer Space features an indignant alien calling the human race, ""...stupid! Stupid, stupid stupid!"" I\'d have to say exhibit A in that trial would probably this movie, a ridiculously silly sci-fi film.<br /><br />Falling action star Jean Claude Van Damme returns to a hit role for him from the original movie, Luke, a former Universal Soldier who now works making really good universal soldiers. While Van Damme was too big to reprise the role in the first two sequels, he was too small to do much of anything else by the time the fourth film in the Universal Soldier series came around. So, probably cursing under his breath the whole way, he kicks and grunts and scowls through ninety minutes of explosions and karate kicks. You\'ll find plenty of mindless violence, but I\'d advise you get a coat check for your brain at the door when you start watching this thing. Otherwise, you are liable to forget where you left it by the time it\'s over.<br 

In [85]:
def  get_mnibatch(doc_stream, size):
    """ 
    permitira obtener subconjuntos, con size que nos dara el tamaño de las secuencias
    que se extraera
    """
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [86]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

#se realiza la vectorizacion con hashingvectorizer
vect=HashingVectorizer(decode_error='ignore', n_features=2**21,
                       preprocessor=None,tokenizer=tokenizer,ngram_range=(1,1))
#creamos una instacion de gradiente en desenso clasificador como logistico "loss=log"
clf=SGDClassifier(loss='log_loss', random_state=1,max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [87]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
# tomare 45 sudconjunto de 1000 cada uno, de los 50000, se dejan 5000 para test
for _ in range(45):
    X_train,y_train = get_mnibatch(doc_stream,size=1000)
    if not X_train:
        break
    X_train =vect.transform(X_train)
    clf.partial_fit(X_train,y_train, classes=classes)
    pbar.update()
clf.score(X_train,y_train)

0% [####################          ] 100% | ETA: 00:00:02

1.0

In [88]:
X_test,y_test = get_mnibatch(doc_stream,size=5000)
X_test=vect.transform(X_test)
clf.score(X_test,y_test)

0.4958

In [89]:
vect.get_params()

{'alternate_sign': True,
 'analyzer': 'word',
 'binary': False,
 'decode_error': 'ignore',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'n_features': 2097152,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.tokenizer(text)>}